# Install libraries

In [14]:
import cv2   # for image and video processing
import time
import os
import numpy as np
import mediapipe as mp    # Framework for building multimodal applied machine learning pipelines
import torch.optim as optim     # For building neural networks,( data loaders and transformations ).
from torchvision import models
import torchmetrics    # A library for various machine learning metrics
from PIL import Image  # Used for opening and saving many different image file formats.
import torch    # unpacking (zip or rar files).
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from pyunpack import Archive
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical   # for utilities like categorizing labels

2024-05-28 18:32:13.864218: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-28 18:32:14.764752: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [13]:
# !pip install mediapipe
# !pip install pyunpack
# !pip install patool
# !sudo apt-get install unrar
# !pip install opencv-python
# !pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 11.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 105.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 93.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 68.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 90.6 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 95.8 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 29.8 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.0
    Unin

In [15]:
import torch
torch.cuda.is_available()

True

# Key points using mp holistic

In [16]:
 # Initializes MediaPipe's holistic model for processing full-body pose, face, hands.
    
mp_holistic = mp.solutions.holistic
mp_drawing=mp.solutions.drawing_utils

In [17]:
 #It converts the image to RGB, processes it, and converts it back to BGR

def mediapipe_detection(image, model):
    image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image.flags.writeable=False
    results=model.process(image)
    image.flags.writeable=True
    image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    return image, results


In [18]:
 # A function to draw the detected landmarks for face, pose, and hands on an image.
    
def draw_landmarks(image,model):
    mp_drawing.draw_landmarks(image, results.face_landmarks,mp_holistic.FACEMESH_TESSELATION)
    mp_drawing.draw_landmarks(image, results.pose_landmarks,mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS)

In [19]:
# adding specific styles and colors to the landmarks for the hands, face, and body. 
def draw_styled_landmarks(img_data,results):
    mp_drawing.draw_landmarks(img_data,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                           mp_drawing.DrawingSpec(color=(255, 0, 255),thickness=2,circle_radius=4), # landmark circles
                           mp_drawing.DrawingSpec(color=(249, 8, 70),thickness=2,circle_radius=2)   # landmark connections
                          )

    mp_drawing.draw_landmarks(img_data,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                           mp_drawing.DrawingSpec(color=(255, 255, 0),thickness=2,circle_radius=4),
                           mp_drawing.DrawingSpec(color=(16, 5, 248),thickness=2,circle_radius=2)
                          )

    mp_drawing.draw_landmarks(img_data,results.face_landmarks,mp_holistic.FACEMESH_TESSELATION,
                           mp_drawing.DrawingSpec(color=(244, 30, 9),thickness=1,circle_radius=1)
                          )
    mp_drawing.draw_landmarks(img_data,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS,
                           mp_drawing.DrawingSpec(color=(93, 4, 4 ),thickness=2,circle_radius=4))

# Extract keypoints values

In [20]:
# Extract keypoints from results landmarks
def extract_keypoints(results):
    pose=np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face=np.array([[res.x,res.y,res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh=np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh=np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose,face,lh,rh])

# Setup folders for collection

In [21]:

ROOT_DIR = "Computational"

subfolders = ["Donia", "Adham"]

actions = np.array(['يفكر','يشم','يشرب','يسمع','يبني','ياكل','مع السلامة','مبروك','ما اسمك',
                    'لو سمحت','كيف حالك','كم عمرك','شكرا','سعيد','اين مكانك',
                    'السلام عليكم','الجلد'])

no_sequences = 30
sequence_length = 30

for subfolder in subfolders:
    DATA_PATH = os.path.join(ROOT_DIR, subfolder)
    print(f"Data path for {subfolder}: {DATA_PATH}")

    for action in actions:
        action_path = os.path.join(DATA_PATH, action)
        print(f"Path for action {action} in {subfolder}: {action_path}")


Data path for Donia: Computational/Donia
Path for action يفكر in Donia: Computational/Donia/يفكر
Path for action يشم in Donia: Computational/Donia/يشم
Path for action يشرب in Donia: Computational/Donia/يشرب
Path for action يسمع in Donia: Computational/Donia/يسمع
Path for action يبني in Donia: Computational/Donia/يبني
Path for action ياكل in Donia: Computational/Donia/ياكل
Path for action مع السلامة in Donia: Computational/Donia/مع السلامة
Path for action مبروك in Donia: Computational/Donia/مبروك
Path for action ما اسمك in Donia: Computational/Donia/ما اسمك
Path for action لو سمحت in Donia: Computational/Donia/لو سمحت
Path for action كيف حالك in Donia: Computational/Donia/كيف حالك
Path for action كم عمرك in Donia: Computational/Donia/كم عمرك
Path for action شكرا in Donia: Computational/Donia/شكرا
Path for action سعيد in Donia: Computational/Donia/سعيد
Path for action اين مكانك in Donia: Computational/Donia/اين مكانك
Path for action السلام عليكم in Donia: Computational/Donia/السلام عليكم

In [22]:
for action in actions:
    for sequence in range (no_sequences):
        try:
             os.makedirs(os.path.join(DATA_PATH,action,str(sequence)))
        except:
            pass

# Collect Keypoints for training and testing

In [7]:
cap=cv2.VideoCapture(0)  # intialize a video webcam

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # min_detection_confidence= min confidence that the model must have to consider a detection successful.
    # min_tracking_confidence = min confidence that the model must have to continue tracking body part.
    
    for action in actions:     # Nested Loops for Data Collection
        for sequence in range(no_sequences):
            for frame_num in range (sequence_length):
                
                #read feed
                ret, frame=cap.read()

                #make detections
                image,results=mediapipe_detection(frame, holistic)
                print(results)

                # draw landmarks
                draw_styled_landmarks(image,results)

                #Apply wait logic
                if frame_num==0:
                    cv2.putText(image,'Starting Collection',(120,200),
                               cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),4,cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} video number {}'.format(actions, sequence), (15, 12),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)

                    cv2.imshow('OpenCv Feed', image)
                    cv2.waitKey(2000)
                else:
                    cv2.putText(image,'Collecting frames for {} video number {}'.format(action,sequence),(15,12),
                               cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)

                    cv2.imshow('OpenCv Feed', image)

                keypoints=extract_keypoints(results)
                npy_path=os.path.join(DATA_PATH,action, str(sequence),str(frame_num))
                np.save(npy_path,keypoints)


                # break gracefully
                if cv2.waitKey(10) & 0xFF==ord('q'):
                    break
    cap.release()
    cv2.destroyAllWindows()

# Preprocessing

In [23]:
label_map={label:num for num, label in enumerate(actions)}

In [24]:
label_map

{'يفكر': 0,
 'يشم': 1,
 'يشرب': 2,
 'يسمع': 3,
 'يبني': 4,
 'ياكل': 5,
 'مع السلامة': 6,
 'مبروك': 7,
 'ما اسمك': 8,
 'لو سمحت': 9,
 'كيف حالك': 10,
 'كم عمرك': 11,
 'شكرا': 12,
 'سعيد': 13,
 'اين مكانك': 14,
 'السلام عليكم': 15,
 'الجلد': 16}

In [27]:
sequences = []
labels = []

# Use separate data paths for each subfolder as needed in your operations
for subfolder in subfolders:
    DATA_PATH = os.path.join(ROOT_DIR, subfolder)
    print(f"Data path for {subfolder}: {DATA_PATH}")

    # Iterate through actions
    for action in actions:
        action_path = os.path.join(DATA_PATH, action)
        print(f"Path for action {action} in {subfolder}: {action_path}")

        # Iterate through sequences
        for sequence in range(no_sequences):
            window = []
            for frame_num in range(sequence_length):
                frame_path = os.path.join(action_path, str(sequence), f"{frame_num}.npy")
                try:
                    res = np.load(frame_path)
                    window.append(res)
                except FileNotFoundError:
                    print(f"File not found: {frame_path}")
                    continue  # Skip this frame if file is not found

            if window:  # Only append to sequences if window is not empty
                sequences.append(window)
                labels.append(label_map[action])

Data path for Donia: Computational/Donia
Path for action يفكر in Donia: Computational/Donia/يفكر
File not found: Computational/Donia/يفكر/0/0.npy
File not found: Computational/Donia/يفكر/0/1.npy
File not found: Computational/Donia/يفكر/0/2.npy
File not found: Computational/Donia/يفكر/0/3.npy
File not found: Computational/Donia/يفكر/0/4.npy
File not found: Computational/Donia/يفكر/0/5.npy
File not found: Computational/Donia/يفكر/0/6.npy
File not found: Computational/Donia/يفكر/0/7.npy
File not found: Computational/Donia/يفكر/0/8.npy
File not found: Computational/Donia/يفكر/0/9.npy
File not found: Computational/Donia/يفكر/0/10.npy
File not found: Computational/Donia/يفكر/0/11.npy
File not found: Computational/Donia/يفكر/0/12.npy
File not found: Computational/Donia/يفكر/0/13.npy
File not found: Computational/Donia/يفكر/0/14.npy
File not found: Computational/Donia/يفكر/0/15.npy
File not found: Computational/Donia/يفكر/0/16.npy
File not found: Computational/Donia/يفكر/0/17.npy
File not foun

In [30]:

def extract_rar(file_path, extract_to):
    Archive(file_path).extractall(extract_to)              #hereeee

# Usage
rar_file_path = 'Computational.rar'
destination_folder = 'Extract'
extract_rar(rar_file_path, destination_folder)


In [31]:

# data handling

class CorrectedImageDataset(Dataset):
    def __init__(self, root_dir, transform=None): # initialize dataset 
        self.root_dir = root_dir # store the directory where the data is located
        self.label_to_index = {'Donia': 0, 'Adham': 1}  # Map string labels to integers
        
        #Resizing image
        self.transform = transform or Compose([  
            Resize((224, 224)),
            ToTensor(),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        self.samples = [] # hold the samples
        
        # Looping on the dataset
        for class_dir in os.listdir(root_dir):
            class_path = os.path.join(root_dir, class_dir)
            for word_dir in os.listdir(class_path):
                word_path = os.path.join(class_path, word_dir)
                for file_dir in os.listdir(word_path):
                    file_path = os.path.join(word_path, file_dir)
                    for frame_file in os.listdir(file_path):
                        if frame_file.endswith('.npy'):
                            frame_path = os.path.join(file_path, frame_file)
                            data = np.load(frame_path)
                            data = (data - np.min(data)) / (np.max(data) - np.min(data))  # Normalize to 0-1
                            try:
                                # Reshape data to (41, 40)
                                data = data[:1640].reshape((41, 40))
                                # Pad the data to make it square
                                data = np.pad(data, ((0, 0), (0, 1)), 'constant', constant_values=0)  # (41, 41)
                                data = np.stack((data, data, data), axis=-1)  # Convert to RGB
                                
                                # Convert class label string to integer
                                class_index = self.label_to_index[class_dir]
                                self.samples.append((data, class_index))
                            except ValueError:
                                print(f"Error reshaping file {frame_path}. Skipping.")

    def __len__(self): # Returns the number of items in the dataset 
        return len(self.samples)

    def __getitem__(self, idx):
        data, class_index = self.samples[idx] #retreive the index of samplees to the data with index 
        data = (255 * data).astype(np.uint8) #change to the standard data type f image in pytorch [0,255]
        image = Image.fromarray(data, 'RGB') 
        image = self.transform(image)
        return image, torch.tensor(class_index)

# Usage of the dataset
dataset = CorrectedImageDataset(root_dir='Extract/Computational')
data_loader = DataLoader(dataset, batch_size=10, shuffle=True)

for images, labels in data_loader:
    print(f"Batch shape: {images.shape}, Batch labels: {labels}")


/tmp/ipykernel_2119/3256568650.py:27: RuntimeWarning: divide by zero encountered in divide
  data = (data - np.min(data)) / (np.max(data) - np.min(data))  # Normalize to 0-1
/tmp/ipykernel_2119/3256568650.py:27: RuntimeWarning: invalid value encountered in divide
  data = (data - np.min(data)) / (np.max(data) - np.min(data))  # Normalize to 0-1


Batch shape: torch.Size([10, 3, 224, 224]), Batch labels: tensor([1, 1, 1, 0, 1, 0, 0, 0, 0, 0])
Batch shape: torch.Size([10, 3, 224, 224]), Batch labels: tensor([1, 0, 1, 1, 1, 1, 0, 0, 0, 1])
Batch shape: torch.Size([10, 3, 224, 224]), Batch labels: tensor([1, 1, 0, 1, 0, 1, 1, 1, 1, 1])
Batch shape: torch.Size([10, 3, 224, 224]), Batch labels: tensor([0, 1, 1, 1, 1, 1, 0, 0, 1, 1])
Batch shape: torch.Size([10, 3, 224, 224]), Batch labels: tensor([0, 1, 1, 0, 0, 0, 1, 0, 1, 0])
Batch shape: torch.Size([10, 3, 224, 224]), Batch labels: tensor([0, 1, 1, 0, 0, 1, 1, 1, 1, 1])
Batch shape: torch.Size([10, 3, 224, 224]), Batch labels: tensor([0, 0, 0, 1, 1, 0, 1, 1, 0, 1])
Batch shape: torch.Size([10, 3, 224, 224]), Batch labels: tensor([1, 0, 1, 0, 0, 1, 0, 0, 1, 0])
Batch shape: torch.Size([10, 3, 224, 224]), Batch labels: tensor([0, 1, 1, 1, 1, 0, 0, 1, 0, 1])
Batch shape: torch.Size([10, 3, 224, 224]), Batch labels: tensor([0, 1, 0, 0, 0, 1, 0, 0, 1, 1])
Batch shape: torch.Size([10, 3

/tmp/ipykernel_2119/3256568650.py:46: RuntimeWarning: invalid value encountered in cast
  data = (255 * data).astype(np.uint8) #change to the standard data type f image in pytorch [0,255]


Batch shape: torch.Size([10, 3, 224, 224]), Batch labels: tensor([0, 1, 0, 1, 1, 1, 0, 1, 1, 0])
Batch shape: torch.Size([10, 3, 224, 224]), Batch labels: tensor([0, 1, 0, 1, 1, 1, 0, 1, 1, 1])
Batch shape: torch.Size([10, 3, 224, 224]), Batch labels: tensor([0, 0, 0, 1, 1, 0, 1, 0, 1, 0])
Batch shape: torch.Size([10, 3, 224, 224]), Batch labels: tensor([0, 1, 0, 1, 1, 0, 1, 1, 1, 1])
Batch shape: torch.Size([10, 3, 224, 224]), Batch labels: tensor([0, 1, 0, 0, 0, 1, 0, 1, 1, 0])
Batch shape: torch.Size([10, 3, 224, 224]), Batch labels: tensor([0, 1, 1, 1, 0, 0, 1, 0, 1, 1])
Batch shape: torch.Size([10, 3, 224, 224]), Batch labels: tensor([1, 1, 0, 1, 0, 0, 1, 1, 0, 1])
Batch shape: torch.Size([10, 3, 224, 224]), Batch labels: tensor([0, 1, 1, 1, 1, 1, 1, 1, 1, 1])
Batch shape: torch.Size([10, 3, 224, 224]), Batch labels: tensor([1, 1, 1, 0, 0, 1, 1, 1, 0, 0])
Batch shape: torch.Size([10, 3, 224, 224]), Batch labels: tensor([0, 1, 1, 1, 1, 0, 1, 1, 0, 1])
Batch shape: torch.Size([10, 3

In [32]:
model = models.vit_b_16(pretrained=True) # 16 = patch size used by the model / 
          #pretrained=True-> specifies that the model should be loaded with weights trained on a large dataset

# Replace the classifier head to fit our number of classes (17 in this case)
model.heads = nn.Sequential(
    nn.Linear(model.heads[0].in_features, 17)  
)

# Check if CUDA is available 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

VisionTransformer(
  (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=3072, out_features=768, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_a

In [33]:

criterion = nn.CrossEntropyLoss()  # loss function commonly used in classification tasks
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Using Adam optimizer
                                                      # Set learning rate to 0.001

num_epochs = 10

model.train()  # Set the model to training mode
for epoch in range(num_epochs):
    total_loss = 0
    for images, labels in data_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to the device

        optimizer.zero_grad()  # Zero the gradients (reset the loss)
        outputs = model(images)  # Forward pass (The model processes images, producing outputs.)
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backward pass (Performs backpropagation from the loss, calculating gradients for each parameter weight)
        optimizer.step()  # Update weights

        total_loss += loss.item()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(data_loader)}')


/tmp/ipykernel_2119/3256568650.py:46: RuntimeWarning: invalid value encountered in cast
  data = (255 * data).astype(np.uint8) #change to the standard data type f image in pytorch [0,255]


Epoch 1/10, Loss: 0.6322109708550319
Epoch 2/10, Loss: 0.35940916370691767
Epoch 3/10, Loss: 0.29163954592252067
Epoch 4/10, Loss: 0.27127114837133975
Epoch 5/10, Loss: 0.2511497987485247
Epoch 6/10, Loss: 0.21569131832148308
Epoch 7/10, Loss: 0.2538870952883431
Epoch 8/10, Loss: 0.25955715095905857
Epoch 9/10, Loss: 0.232019574597274
Epoch 10/10, Loss: 0.20697909582716723


In [36]:
# Evaluate the model
model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Turn off gradients for validation, saves memory and computations
    correct = 0
    total = 0
    for images, labels in data_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1) #function returns the indices of the maximum values, which represent the predicted class labels.
        total += labels.size(0)
        correct += (predicted == labels).sum().item() # calculates the number of correct predictions in the batch by comparing predicted and labels

    print(f'Accuracy on validation set: {100 * correct / total}%')


/tmp/ipykernel_2119/3256568650.py:46: RuntimeWarning: invalid value encountered in cast
  data = (255 * data).astype(np.uint8) #change to the standard data type f image in pytorch [0,255]


Accuracy on validation set: 89.69934640522875%


In [37]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Metrics initialization with the task and number of classes specified, and moving them to the correct device
accuracy = torchmetrics.Accuracy(num_classes=17, average='macro', task='multiclass').to(device)
precision = torchmetrics.Precision(num_classes=17, average='macro', task='multiclass').to(device)
recall = torchmetrics.Recall(num_classes=17, average='macro', task='multiclass').to(device)
f1 = torchmetrics.F1Score(num_classes=17, average='macro', task='multiclass').to(device)

model.eval()
with torch.no_grad():
    for images, labels in data_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        preds = torch.argmax(outputs, dim=1)
        
        # Update metrics
        accuracy.update(preds, labels)
        precision.update(preds, labels)
        recall.update(preds, labels)
        f1.update(preds, labels)

    # Compute final metric scores
    final_accuracy = accuracy.compute()
    final_precision = precision.compute()
    final_recall = recall.compute()
    final_f1 = f1.compute()

    print(f'Validation Accuracy: {final_accuracy:.4f}')
    print(f'Validation Precision: {final_precision:.4f}')
    print(f'Validation Recall: {final_recall:.4f}')
    print(f'Validation F1 Score: {final_f1:.4f}')


/tmp/ipykernel_2119/3256568650.py:46: RuntimeWarning: invalid value encountered in cast
  data = (255 * data).astype(np.uint8) #change to the standard data type f image in pytorch [0,255]


Validation Accuracy: 0.8970
Validation Precision: 0.8970
Validation Recall: 0.8970
Validation F1 Score: 0.8970


In [38]:
outputs

tensor([[ -0.8956,   1.3409, -12.6877, -12.6608, -12.6761, -12.7848, -12.9330,
         -12.5143, -12.9022, -12.6791, -12.5141, -12.7869, -12.7398, -12.9691,
         -12.8533, -12.7647, -12.4880],
        [ -1.9103,   2.4813, -12.2391, -12.2291, -12.2314, -12.3451, -12.4474,
         -12.0810, -12.4092, -12.2347, -12.0871, -12.3003, -12.2801, -12.4934,
         -12.3882, -12.3051, -12.0812],
        [  2.2173,  -2.3361, -13.0603, -12.9706, -13.0319, -13.1315, -13.3224,
         -12.8680, -13.3233, -13.0441, -12.8270, -13.1838, -13.1276, -13.3508,
         -13.2376, -13.1404, -12.7919],
        [  0.4273,  -0.1580, -13.4971, -13.4345, -13.4577, -13.5686, -13.6610,
         -13.3217, -13.6266, -13.4812, -13.3018, -13.5202, -13.5208, -13.7105,
         -13.6330, -13.5203, -13.3124],
        [ -1.9641,   2.5390, -12.2437, -12.2270, -12.2245, -12.3436, -12.4069,
         -12.0902, -12.3647, -12.2503, -12.1063, -12.2598, -12.2641, -12.4734,
         -12.3734, -12.2865, -12.1081],
        [ 

In [39]:
preds

tensor([1, 1, 0, 0, 1, 1, 0, 0, 0, 0], device='cuda:0')

In [40]:
labels

tensor([1, 1, 0, 1, 1, 1, 0, 0, 0, 0], device='cuda:0')

In [44]:

# Print predictions
for i, preds in enumerate(outputs):
    print(f"Sample {i}: Class {preds}")


Sample 0: Class tensor([ -0.8956,   1.3409, -12.6877, -12.6608, -12.6761, -12.7848, -12.9330,
        -12.5143, -12.9022, -12.6791, -12.5141, -12.7869, -12.7398, -12.9691,
        -12.8533, -12.7647, -12.4880], device='cuda:0')
Sample 1: Class tensor([ -1.9103,   2.4813, -12.2391, -12.2291, -12.2314, -12.3451, -12.4474,
        -12.0810, -12.4092, -12.2347, -12.0871, -12.3003, -12.2801, -12.4934,
        -12.3882, -12.3051, -12.0812], device='cuda:0')
Sample 2: Class tensor([  2.2173,  -2.3361, -13.0603, -12.9706, -13.0319, -13.1315, -13.3224,
        -12.8680, -13.3233, -13.0441, -12.8270, -13.1838, -13.1276, -13.3508,
        -13.2376, -13.1404, -12.7919], device='cuda:0')
Sample 3: Class tensor([  0.4273,  -0.1580, -13.4971, -13.4345, -13.4577, -13.5686, -13.6610,
        -13.3217, -13.6266, -13.4812, -13.3018, -13.5202, -13.5208, -13.7105,
        -13.6330, -13.5203, -13.3124], device='cuda:0')
Sample 4: Class tensor([ -1.9641,   2.5390, -12.2437, -12.2270, -12.2245, -12.3436, -12.

In [46]:
preds.shape

torch.Size([17])